In [1]:
from aocd import get_data
data = get_data(day=11, year=2023)

In [1]:
data = '''...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....'''

In [2]:
import numpy as np
from scipy import ndimage

In [23]:
m = np.array([ list(x) for x in data.splitlines() ])
for y, row in enumerate(m):
    if all(row == '.'):
        m = np.insert(m, y, '.', axis=0)
for x, col in enumerate(m.T):
    if all(col == '.'):
        m = np.insert(m, x, '.', axis=1)

m = np.where(m == '#', 1, 0)
labeled, lcount = ndimage.label(m)
labeled

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
       [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0],
       [0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0],
       [8, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [17]:
from scipy.spatial.distance import cdist

def feature_dist(input):
    """
    Takes a labeled array as returned by scipy.ndimage.label and 
    returns an intra-feature distance matrix.
    """
    I, J = np.nonzero(input)
    labels = input[I,J]
    coords = np.column_stack((I,J))

    sorter = np.argsort(labels)
    labels = labels[sorter]
    coords = coords[sorter]

    sq_dists = cdist(coords, coords, 'sqeuclidean')

    start_idx = np.flatnonzero(np.r_[1, np.diff(labels)])
    nonzero_vs_feat = np.minimum.reduceat(sq_dists, start_idx, axis=1)
    feat_vs_feat = np.minimum.reduceat(nonzero_vs_feat, start_idx, axis=0)

    return np.sqrt(feat_vs_feat)

In [22]:
feature_dist(labeled)

array([[ 0.        ,  6.08276253,  4.47213595,  7.07106781,  6.70820393,
        11.3137085 , 11.66190379, 11.70469991, 11.18033989],
       [ 6.08276253,  0.        , 10.04987562,  4.12310563, 10.29563014,
         7.28010989,  9.        , 14.14213562, 10.77032961],
       [ 4.47213595, 10.04987562,  0.        ,  9.48683298,  4.12310563,
        13.41640786, 12.80624847,  9.        , 10.81665383],
       [ 7.07106781,  4.12310563,  9.48683298,  0.        ,  8.06225775,
         4.24264069,  5.09901951, 10.81665383,  6.70820393],
       [ 6.70820393, 10.29563014,  4.12310563,  8.06225775,  0.        ,
        11.18033989,  9.8488578 ,  5.09901951,  7.07106781],
       [11.3137085 ,  7.28010989, 13.41640786,  4.24264069, 11.18033989,
         0.        ,  2.82842712, 12.36931688,  6.70820393],
       [11.66190379,  9.        , 12.80624847,  5.09901951,  9.8488578 ,
         2.82842712,  0.        , 10.04987562,  4.12310563],
       [11.70469991, 14.14213562,  9.        , 10.81665383,  5